In [2]:
from typing import Iterator
import numpy as np
import os
import torch
from problog.logic import Term, Constant, list2term
from deepproblog.dataset import Dataset
from deepproblog.query import Query
import matplotlib.pyplot as plt
import sys

In [3]:
import torch
import torch.nn as nn

class MNIST_SudokuNet(nn.Module):
    def __init__(self):
        super(MNIST_SudokuNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.MaxPool2d(2, 2),
            nn.ReLU(True),
            nn.Conv2d(6, 16, 5),
            nn.MaxPool2d(2, 2),
            nn.ReLU(True),
        )
        # Only 9 outputs now
        self.classifier = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 9),   # 9 classes => digits 1..9
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        x = self.encoder(x)                # 1×28×28 -> ...
        x = x.view(-1, 16 * 4 * 4)         
        x = self.classifier(x)             # -> 9 outputs 
        return x


In [4]:
class SudokuPuzzleDataset(Dataset):
    def __init__(self, subset: str, puzzle_path: str, solution_path: str):
        self.subset = subset
        self.puzzle_files = sorted([f for f in os.listdir(puzzle_path) if f.endswith('.npy')])
        self.puzzles = []
        for fn in self.puzzle_files:
            puzzle = np.load(os.path.join(puzzle_path, fn), allow_pickle=True)
            # Keep puzzle[i][j] == None if empty
            # If not None, we treat it as a 28×28 image
            #   and normalize it (1..255 => /255)
            for i in range(9):
                for j in range(9):
                    if puzzle[i][j] is not None:
                        puzzle[i][j] = puzzle[i][j].astype(np.float32)/255.0
            self.puzzles.append(puzzle)

        self.solution_files = sorted(f for f in os.listdir(solution_path) if f.endswith('.npy'))
        self.solutions = []
        for fn in self.solution_files:
            solution = np.load(os.path.join(solution_path, fn), allow_pickle=True)
            self.solutions.append(solution)

    def __len__(self):
        return len(self.puzzles)

    def __getitem__(self, item):
        """
        For the Prolog side, we only need to retrieve the actual image data 
        if the cell is not None. We'll handle that in 'to_query' as well.
        """
        # This method is used by the DeepProbLog model to fetch the image.
        # We'll just return a single cell's tensor to illustrate 
        # (not typically used if you do full puzzle queries).
        puzzle_index, row, col = item
        cell = self.puzzles[int(puzzle_index)][int(row)][int(col)]
        if cell is None:
            # Not returning an image for None cells
            raise ValueError("Attempted to retrieve an image for a None cell.")
        # print
        return torch.tensor(cell).unsqueeze(0)  # 1×28×28

    def to_query(self, index: int) -> Query:
        """
        Construct a query puzzle_solve(...) for the entire 9×9 puzzle.
        We'll produce a 9×9 list-of-lists:
          - If puzzle[i][j] is None => use Constant('none')
          - Else => Term('tensor', Term(self.subset, Constant(unique_id)))
        """
        puzzle = self.puzzles[index]
        # We'll create 9 rows of terms
        row_list_terms = []

        for i in range(9):
            col_terms = []
            for j in range(9):
                if puzzle[i][j] is None:
                    # Just an atom 'none'
                    col_terms.append(Constant('none'))
                else:
                    # A reference to the network input
                    image_term = Term("tensor", Term(self.subset, Constant(index), Constant(i), Constant(j)))
                    label = Constant(self.solutions[index][i][j])
                    col_terms.append(Term("digit", image_term, label))

            row_list_terms.append(list2term(col_terms))

        # puzzle_solve( [[...],[...],...] )
        query_term = Term("puzzle_solve", list2term(row_list_terms))
        # print("Query constructed", query_term)
        return Query(query_term)

In [6]:
import torch
from deepproblog.model import Model
from deepproblog.network import Network
from deepproblog.engines import ExactEngine
from deepproblog.dataset import DataLoader
from deepproblog.train import train_model

In [ ]:
network = MNIST_SudokuNet()
net = Network(network, "mnist_net", batching=True)
net.optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)

# 2) Create the model from 'sudoku_no_clpfd.pl'
model = Model("prolog3.pl", [net])
model.set_engine(ExactEngine(model))

# 3) Load the dataset
train_data = SudokuPuzzleDataset("train", "../../mnist_sudoku_generator/dataset/images/puzzles/train", "../../mnist_sudoku_generator/dataset/arrays/puzzles/train")
model.add_tensor_source("train", train_data)

loader = DataLoader(train_data, batch_size=1, shuffle=True)
train_model(model, loader, 1, log_iter=1)

Training  for 1 epoch(s)
Epoch 1


1 "puzzle_solve/1: Called with ImageRows=~w" [[[none, digit(tensor(train(1904,0,1)),8), digit(tensor(train(1904,0,2)),6), digit(tensor(train(1904,0,3)),3), none, digit(tensor(train(1904,0,5)),1), none, none, digit(tensor(train(1904,0,8)),9)], [none, digit(tensor(train(1904,1,1)),4), digit(tensor(train(1904,1,2)),2), none, digit(tensor(train(1904,1,4)),5), digit(tensor(train(1904,1,5)),7), digit(tensor(train(1904,1,6)),8), none, none], [digit(tensor(train(1904,2,0)),5), none, none, digit(tensor(train(1904,2,3)),9), digit(tensor(train(1904,2,4)),4), digit(tensor(train(1904,2,5)),8), none, digit(tensor(train(1904,2,7)),6), digit(tensor(train(1904,2,8)),7)], [digit(tensor(train(1904,3,0)),4), digit(tensor(train(1904,3,1)),5), digit(tensor(train(1904,3,2)),7), none, digit(tensor(train(1904,3,4)),1), digit(tensor(train(1904,3,5)),3), digit(tensor(train(1904,3,6)),9), none, digit(tensor(train(1904,3,8)),6)], [digit(tensor(train(1904,4,0)),8), none, none, none, none, digit(tensor(train(1904,4,

Iteration:  2 	s:190.1980 	Average Loss:  27.63102149963379


1 "puzzle_solve/1: Called with ImageRows=~w" [[[none, digit(tensor(train(5325,0,1)),1), none, none, digit(tensor(train(5325,0,4)),7), none, none, none, none], [none, none, digit(tensor(train(5325,1,2)),9), digit(tensor(train(5325,1,3)),3), digit(tensor(train(5325,1,4)),6), digit(tensor(train(5325,1,5)),1), none, digit(tensor(train(5325,1,7)),2), digit(tensor(train(5325,1,8)),5)], [none, none, none, digit(tensor(train(5325,2,3)),8), digit(tensor(train(5325,2,4)),5), digit(tensor(train(5325,2,5)),9), none, digit(tensor(train(5325,2,7)),3), none], [digit(tensor(train(5325,3,0)),5), digit(tensor(train(5325,3,1)),8), none, none, none, none, digit(tensor(train(5325,3,6)),3), digit(tensor(train(5325,3,7)),1), digit(tensor(train(5325,3,8)),9)], [none, none, none, none, none, digit(tensor(train(5325,4,5)),4), digit(tensor(train(5325,4,6)),2), none, none], [digit(tensor(train(5325,5,0)),9), digit(tensor(train(5325,5,1)),2), digit(tensor(train(5325,5,2)),6), digit(tensor(train(5325,5,3)),1), none

Iteration:  3 	s:87.5221 	Average Loss:  27.63102149963379


...

Iteration:  13 	s:238.0315 	Average Loss:  27.63102149963379


1 "transpose/2: Transposed ~w" [[[X1, X2, 6, 7, 9, X3, 8, X4, X5], [1, X6, 2, 3, X7, 6, X8, 7, X9], [9, X10, 7, 8, X11, X12, X13, 1, X14], [X15, 6, 9, 5, X16, X17, 1, X18, 3], [5, X19, X20, 1, X21, X22, X23, 6, X24], [X25, X26, X27, 6, 7, 9, 5, 8, X28], [6, 7, X29, 9, X30, X31, 2, X32, X33], [3, 9, X34, X35, X36, 7, 6, X37, 8], [8, 2, X38, X39, 6, 3, X40, 9, 5]]]
1 "getBoxes/4: Called with H=~w, T=~w, BoxWidth=~w, BoxHeight=~w" [[X1, 1, 9, X2, 5, X3, 6, 3, 8], [[X4, X5, X6, 6, X7, X8, 7, 9, 2], [6, 2, 7, 9, X9, X10, X11, X12, X13], [7, 3, 8, 5, 1, 6, 9, X14, X15], [9, X16, X17, X18, X19, 7, X20, X21, 6], [X22, 6, X23, X24, X25, 9, X26, 7, 3], [8, X27, X28, 1, X29, 5, 2, 6, X30], [X31, 7, 1, X32, 6, 8, X33, X34, 9], [X35, X36, X37, 3, X38, X39, X40, 8, 5]], 3, 3]
1 "getSolution/2: Sudoku solved, but the garbage persists." []
1 "printing solution" []
1 "" [[4, 1, 9, 7, 5, 2, 6, 3, 8]]
1 "" [[3, 8, 5, 6, 4, 1, 7, 9, 2]]
1 "" [[6, 2, 7, 9, 8, 3, 4, 5, 1]]
1 "" [[7, 3, 8, 5, 1, 6, 9, 2, 4]]

Iteration:  14 	s:220.3534 	Average Loss:  27.63102149963379


1 "transpose/2: Transposed ~w" [[[X1, 8, X2, 3, X3, 5, X4, X5, 4], [X6, X7, X8, 2, 9, X9, X10, 8, 3], [9, X11, X12, 7, 8, X13, 1, X14, 6], [X15, X16, 8, X17, 6, X18, 2, 3, X19], [X20, 1, 9, X21, X22, 3, 6, 4, 5], [3, X23, 6, 5, X24, X25, 9, X26, 8], [X27, 9, X28, 6, X29, X30, 4, 7, 2], [5, X31, 4, 1, 3, X32, 8, 6, X33], [X34, X35, 7, X36, X37, X38, 3, X39, X40]]]
1 "getBoxes/4: Called with H=~w, T=~w, BoxWidth=~w, BoxHeight=~w" [[X1, X2, 9, X3, X4, 3, X5, 5, X6], [[8, X7, X8, X9, 1, X10, 9, X11, X12], [X13, X14, X15, 8, 9, 6, X16, 4, 7], [3, 2, 7, X17, X18, 5, 6, 1, X19], [X20, 9, 8, 6, X21, X22, X23, 3, X24], [5, X25, X26, X27, 3, X28, X29, X30, X31], [X32, X33, 1, 2, 6, 9, 4, 8, 3], [X34, 8, X35, 3, 4, X36, 7, 6, X37], [4, 3, 6, X38, 5, 8, 2, X39, X40]], 3, 3]
1 "getSolution/2: Sudoku solved, but the garbage persists." []


KeyboardInterrupt: 

In [9]:
# 4) Inference on a single puzzle
test_query = dataset.to_query(0)
result = model.solve([test_query])
print("Puzzle solution:", result)

Puzzle solution: [{digit(tensor(train_single_image(0)),5): tensor(0.8645, grad_fn=<SelectBackward0>)}]


1 "digit/2 => NN recognized => D=~w" 5
